## . 목표! : 
- 타이타닉 데이터를 활용하여 생존자와 사망자를 예측해보자
- kaggle 경진대회에 참가하여 우리 점수를 확인해보자!
- 머신러닝의 전체과정을 체험해보자

## . 머신러닝 과정
1. 문제정의: 목표 설정, 어떤 모델
2. 데이터 수집: 분류를 할거라면 class로 담긴 레이블 있는 데이터 수집
3. 데이터 전처리 : 머신러닝의 학습을 위해 이상치나 결측치를 처리하는 과정
4. 데이터 탐색: 기술통계, 특성간의 관계
5. 모델 선택 및 하이퍼 파라미터 설정
6. 모델 예측 및 평가
7. 모델 서비스 화

## . 임포트하기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns # 시각화 라이브러리
# train, test 분리하는 도구
from sklearn.model_selection import train_test_split
# 평가 도구
from sklearn.metrics import accuracy_score
# tree model - 사망/ 생존 이진분류
from sklearn.tree import DecisionTreeClassifier

In [2]:
# train, test 변수에 데이터 불러와서 담기
# 단, 인덱스를 승객의 번호로 설정해서 담아줄 것

In [3]:
train = pd.read_csv("data/titanic/train.csv", index_col="PassengerId")
test = pd.read_csv("data/titanic/test.csv", index_col= "PassengerId")

 0.   Survived  - 생존/ 사망 여부 (답 데이터, Class Label)  
 1.   Pclass    - 승객의 등급(1- 귀족층,2- 중간층,3- 하위계층)  
 2.   Name      - 이름        
 3.  Sex       - 성별(male, female)
 4.  Age       - 나이(0~  
 5.  SibSp     - 형제자매+배우자 수  
 6.  Parch     - 부모 + 자식 수  
 7.  Ticket    - 티겟 번호  
 8.  Fare      - 요금  
 9.  Cabin     - 객실번호  
 10.  Embarked  - 승선항  

In [4]:
test.info() # 테스트 정보확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


In [5]:
train.info() # 정보 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [6]:
# 크기 확인
print('트레인 크기: ', train.shape)
print('테스트 크기: ', test.shape) # 테스트에는 정답 데이터가 없다.

트레인 크기:  (891, 11)
테스트 크기:  (418, 10)


In [7]:
# 컬럼명만 뽑아서 확인하기

In [8]:
print(train.columns)
print(test.columns)

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')


### 데이터 전처리 및 탐색
- 결측치 확인

In [9]:
train.info()
#AGE, Cabin , Embarked 에 결측치가 존재

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [10]:
test.count()
#Age에 결측치가 존재

Pclass      418
Name        418
Sex         418
Age         332
SibSp       418
Parch       418
Ticket      418
Fare        417
Cabin        91
Embarked    418
dtype: int64

In [11]:
# 판다스 함수 통해서 test 결측치 확인
# 데이터 자리에 결측치가 있니? => 데이터.isnull()
print(test.shape)
test.isnull().sum()
# 결측치 존재 컬럼 Age, Fare, Cabin
# 결측치를 삭제하기 보다는 채우면서 처리해보겠음.

(418, 10)


Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

#### train- Embarked 결측치 처리
- 승객이 탑승한 항구 이름

In [12]:
# trian 데이터를 기준으로 embarked 컬럼 확인

In [13]:
train['Embarked']

PassengerId
1      S
2      C
3      S
4      S
5      S
      ..
887    S
888    S
889    S
890    C
891    Q
Name: Embarked, Length: 891, dtype: object

In [14]:
#test 안보나요
# 탐색용 데이터는 trian,
# 모델학습 : 모델이 학습을 잘하도록 데이터를 셋팅
# test 데이터는 모델을 일반화 하는데 도움이 되지 않음
# train 데이터가 학습할 때 사용되고 모델의 일반화에 도움을 줌

In [15]:
# Embarekd 결측치를 'S'로 채우기
#fillna(채울값)
train['Embarked'] = train['Embarked'].fillna('S')
train['Embarked'].unique()
train['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [16]:
# 자동화 하는 코드를 짜면 되겟넹

####  test-fare데이터 결측치 처리
1. 요금 평균, 중앙값 계산 가능
2. 데이터 분석을 진행하면서 관련이 있을 법한 컬럼을 연관지어서 대체값을 확인해보자

In [17]:
# train을 기준으로 데이터 탐색
train['Fare'].describe()
# 4분위 수와 중앙값 , 표준편차, 평균이 나옴
#요금의 대표값으로 평균보다 중앙값이 더 나을 수 있음.

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [24]:
# 변수간의 상관관계
# 상관계수 : - 1 ~ 1 사이의 상관계수
# 각 컬럼의 상관 정도를 파악 할 수 있음.
# -1과 1에 가까울 수록 변수간의 관계성이 있음.
#생존사망과 연관이 있는 컬럼 확인해보기
print(train.corr()['Survived'].abs()) # Pclass가 생존 / 사망에 연관이 있을 수 있겟다.
# 등급에 따라서 요금의 중앙값으로 의미가 있을 수 있다.

Survived    1.000000
Pclass      0.338481
Age         0.077221
SibSp       0.035322
Parch       0.081629
Fare        0.257307
Name: Survived, dtype: float64


C:\Users\user\AppData\Local\Temp\ipykernel_14288\452811239.py:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(train.corr()['Survived'].abs()) # Pclass가 생존 / 사망에 연관이 있을 수 있겟다.


In [25]:
Pclass_coulmn =train['Pclass'].unique()
Pclass_coulmn

array([3, 1, 2], dtype=int64)

In [20]:
# Pclass 와 Fare 두개의 컬럼만 인덱싱하기
pf =train[['Pclass', 'Fare']]
# Pcalss 1,2,3 등급끼리 묶기 (그룹화 하기)
# median() 집계함수 연결

In [21]:
# 불리언 색인으로 찾기 .. 개더러움
pf1 = pf[pf['Pclass'] == 1]
pf2 = pf[pf['Pclass'] == 2]
pf3 = pf[pf['Pclass'] == 3]
mp1 =pf1['Fare'].median()
mp2 =pf2['Fare'].median()
mp3 =pf3['Fare'].median()

In [22]:
answer = pd.DataFrame([mp1,mp2,mp3])
answer.index = answer.index +1 
answer.index.name = "Pclass"
answer.columns = ["Fare"]
answer

,Fare
Pclass,
1,60.2875
2,14.2500
3,8.0500


In [23]:
pf.groupby(['Pclass']).median()

,Fare
Pclass,
1,60.2875
2,14.2500
3,8.0500
